# Importações

In [2]:
import os
import glob
import PyPDF2
import re
import nltk
nltk.download('punkt')
from transformers import AutoTokenizer
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from typing import List

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ricardoronsoni/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Configurações

In [3]:
# Diretório com os arquivos PDF de PCDT
diretorio_pdf = "./pcdt"

# Quantidade máxima de tokens no embedding
qtd_max_tokens = 512

# Sobreposição de frases entre embeddings
sobreposicao_frases = 1

# String para determinar a partir de qual parte o PCDT não terá mais o seu texto carregado
ignorar_restante_pcdt = "REFERÊNCIAS"

# Padrão Regex para recuperar o nome do PCDT na primeira página do documento
padrao_nome_pcdt = r"Terapêuticas – (.*?)\."

# Titulo dos tópicos dos PCDT
titulo_topicos = ["METODOLOGIA",
    "INTRODUÇÃO",
    "CLASSIFICAÇÃO ESTATÍSTICA INTERNACIONAL DE DOENÇAS",
    "DIAGNÓSTICO",
    "CRITÉRIOS DE INCLUSÃO",
    "CRITÉRIOS DE EXCLUSÃO",
    "CASOS ESPECIAIS",
    "TRATAMENTO",
    "FÁRMACOS",
    "ESQUEMAS DE ADMINISTRAÇÃO",
    "TEMPO DE TRATAMENTO",
    "BENEFÍCIOS ESPERADOS",
    "MONITORIZAÇÃO",
    "REGULAÇÃO/CONTROLE/AVALIAÇÃO PELO GESTOR",
    "TERMO DE ESCLARECIMENTO E RESPONSABILIDADE"]

# Projeto do modelo de embedding
modelo = "intfloat/multilingual-e5-large"

# Diretório cache modelos
diretorio_cache_modelo = "./llm/"

# Importar arquivos PDF

In [4]:
# Listar todos os arquivos PDF no diretório
arquivos_pdf = glob.glob(f"{diretorio_pdf}/*.pdf")

# Extrair apenas o nome dos arquivos
nomes_arquivos_pdf = [os.path.basename(arquivo) for arquivo in arquivos_pdf]

# Listar os arquivos PDF localizados
print("Arquivos localizados para processamento:")
for i, nome_arquivo in enumerate(nomes_arquivos_pdf, start=1):
    print(f"{i}- {nome_arquivo}")

Arquivos localizados para processamento:
1- Síndrome de Turner.pdf
2- Doença de Alzheimer.pdf


# Extrair texto PDF

In [5]:
# Obter o nome do PCDT na primeira página
def obter_nome_pcdt(texto):
    resultado = ""
    texto = texto.replace("\n", " ")
    resultado = re.findall(padrao_nome_pcdt, texto)

    # Tratamento para PDF com textos distintos na primeira página
    if len(resultado) > 1:
        nome_pcdt = str(resultado[0]).strip()
    else:
        nome_pcdt = str(resultado)[2:-2].strip()

    nome_pcdt = " ".join(resultado[0].split())

    return nome_pcdt

# Ler os PDF e extrair os conteúdos
def extrair_texto_pdf(pdf):
    texto = ""
    nome_pcdt = ""
    leitor_pdf = PyPDF2.PdfReader(pdf)

    # Passar por todas as páginas de um arquivo pdf
    for i, pagina in enumerate(leitor_pdf.pages):
        texto_pagina = pagina.extract_text()
        texto_higienizado = " ".join(texto_pagina.split())
        
        # Se for a primeira página irá verificar o nome do PCDT
        if i == 0:
            nome_pcdt = obter_nome_pcdt(texto_higienizado)

        # Parar a leitura do PCDT se localizar a string de parada
        if ignorar_restante_pcdt in texto_higienizado:
            indice_ignorar = texto_higienizado.index(ignorar_restante_pcdt)
            texto_higienizado = texto_higienizado[:indice_ignorar + len(ignorar_restante_pcdt)]

        texto += texto_higienizado

        if ignorar_restante_pcdt in texto_pagina:
            break

    return nome_pcdt, texto

texto_extraido = []
for pdf in arquivos_pdf:
    nome_pcdt, texto = extrair_texto_pdf(pdf)
    texto_extraido.append({nome_pcdt: texto})

# Exibir os conteúdos dos PCDT lidos. Textos truncados com 250 caracteres para preservar a visualização do notebook
print("Textos extraídos:")
for item in texto_extraido:
    for chave, valor in item.items():
        print(f"{chave}: {valor[:250] + '...'}")

Textos extraídos:
Síndrome de Turner: MINISTÉRIO DA SAÚDE SECRETARIA DE ATENÇÃ O À SAÚDE SECRETARIA DE CIÊNCI A, TECNOLOGIA E INSU MOS ESTRATÉGICOS PORTARIA CONJUNTA Nº 15, DE 09 DE MAIO DE 201 8 Aprova o Protocolo Clínico e Diretrizes Terapêuticas da Síndrome de Turner. O SECRETÁRIO DE ...
Doença de Alzheimer: MINISTÉRIO DA SAÚDE SECRETARIA DE ATENÇÃO À SAÚDE PORTARIA CONJUNTA Nº 13, DE 28 DE NOVEMBRO DE 2017. Aprova o Protocolo Clínico e Diretrizes Terapêuticas da Doença de A lzheimer. . O SECRETÁRIO DE ATENÇÃO À SAÚDE e o SECRETÁRIO DE CIÊNCIA , TECNOLOG...


# Criar embeddings

In [6]:
# Identificar a posição de início de cada tópico no PCDT
def localizar_topicos(texto_pcdt):
        # Armazena o dicionário contendo {"nome_topico": "posição_início"}
        posicao_titulos = {}

        for nome_titulo in titulo_topicos:
            match = re.search(nome_titulo, texto_pcdt)
            if match:
                posicao_titulos[nome_titulo] = match.start()

        # Ordena a posição dos topicos conforme a sequência lógica dos PCDTs
        posicao_titulos = dict(sorted(posicao_titulos.items(), key=lambda x: x[1]))

        return posicao_titulos
        

# Contar a quantidade de tokens em cada frase
def contar_tokens(frase):
    # Instanciar o tokenizer do modelo
    tokenizer = AutoTokenizer.from_pretrained(diretorio_cache_modelo+"intfloat_multilingual-e5-large")
    tokens = tokenizer(frase)
    quantidade_tokens = len(tokens["input_ids"])

    return quantidade_tokens


# Montar a estutura com o texto e os metadados do embedding
def montar_embedding(nome_pcdt, texto_embedding, lista_embedding_topico, frases_sobreposicao, nome_topico):
    texto_embedding_string = " ".join(texto_embedding)

    # Verifica se é o primeiro embedding do tópico para personalizar o texto do embedding
    if len(lista_embedding_topico) > 0:
        embedding = {
            "topic_content": f"query: Protocolo Clínico e Diretrizes Terapêuticas (PCDT) de {nome_pcdt}. {nome_topico}. {frases_sobreposicao} {texto_embedding_string}",
            "metadata": {
                "source": nome_pcdt,
                "topic": nome_topico
            }
        }
    else: 
        embedding = {
            "topic_content": f"query: Protocolo Clínico e Diretrizes Terapêuticas (PCDT) de {nome_pcdt}. {texto_embedding_string}",
            "metadata": {
                "source": nome_pcdt,
                "topic": nome_topico
            }
        }

    # Cria as frases que serão utilizadas posteriormente para sobreposição
    frases_sobreposicao = texto_embedding[-sobreposicao_frases:]
    frases_sobreposicao = " ".join(frases_sobreposicao)

    return embedding, frases_sobreposicao


# Criar os embeddings a partir das frases extraídas do tópico
def criar_embedding_topico(frases_com_tokens, nome_pcdt, nome_topico):
    lista_embedding_topico = []
    total_tokens_utilizados = 0
    texto_embedding = []
    frases_sobreposicao = ""

    # Ajusta a quantidade máxima de tokens que o tópico poderá ter, tendo em vista os textos que serão adicionados em cada embedding
    qtd_tokens_util = qtd_max_tokens - contar_tokens(f"query: Protocolo Clínico e Diretrizes Terapêuticas (PCDT) de {nome_pcdt}. {nome_topico}.")

    for i, tupla in enumerate(frases_com_tokens):
        frase, num_tokens = tupla

        tokens_sobreposicao = contar_tokens(frases_sobreposicao)

        # Verifica se a frase cabe no embedding
        if num_tokens + tokens_sobreposicao + total_tokens_utilizados <= qtd_tokens_util:
            texto_embedding.append(frase)
            total_tokens_utilizados += num_tokens

        # Se a frase não couber no embedding então o embedding é criado sem a frase atual
        elif num_tokens + tokens_sobreposicao + total_tokens_utilizados > qtd_tokens_util:
            embedding, frases_sobreposicao = montar_embedding(nome_pcdt, texto_embedding, lista_embedding_topico, frases_sobreposicao, nome_topico)                
            lista_embedding_topico.append(embedding)

            total_tokens_utilizados = num_tokens
            texto_embedding = []
            texto_embedding.append(frase)

        # Se for a última frase do tópico então gera o embeddigng
        if i == len(frases_com_tokens) - 1: 
            embedding, frases_sobreposicao = montar_embedding(nome_pcdt, texto_embedding, lista_embedding_topico, frases_sobreposicao, nome_topico) 
            lista_embedding_topico.append(embedding)

    return lista_embedding_topico


lista_embeddings = []

# Iterar sobre os textos extraídos dos PDF
for item_pcdt in texto_extraido:  

    # Iterar sobre um PCDT
    for nome_pcdt, texto_pcdt in item_pcdt.items():
        topicos_localizados = localizar_topicos(texto_pcdt)

        # Variável para receber o texto de cada tópico
        texto_topicos = {}

        # Iterar sobre um tópico de um PCDT
        for i, (nome_topico, posicao_inicio) in enumerate(topicos_localizados.items()):  

            # Tratamento para evitar erro no último tópico
            if i + 1 < len(topicos_localizados):
                inicio_proximo_topico = list(topicos_localizados.values())[i + 1]
            else:
                inicio_proximo_topico = len(texto_pcdt)

            texto_topico = texto_pcdt[posicao_inicio:inicio_proximo_topico].strip()

            # Quebrar o texto do tópico em frases
            frases_topico = nltk.sent_tokenize(texto_topico)
            frases_com_tokens = [(frase, contar_tokens(frase)) for frase in frases_topico]

            # Remover as frases com menos de 15 caracteres
            frases_com_tokens = [(frase, tamanho) for frase, tamanho in frases_com_tokens if tamanho >= 15]

            embeddings_topico = criar_embedding_topico(frases_com_tokens, nome_pcdt, nome_topico)
            lista_embeddings.append(embeddings_topico)

# Textos truncados com 250 caracteres para preservar a visualização do notebook
print(f"{lista_embeddings[:250]} ...")


[[{'topic_content': 'query: Protocolo Clínico e Diretrizes Terapêuticas (PCDT) de Síndrome de Turner. INTRODUÇÃO A síndrome de Turner é a anormalidade dos cromossomos sexuais mais comum nas mulheres, ocorrendo em 1 a cada 1.500 -2.500 crianças do sexo feminino nascidas vivas (1). A constituição cromossômica pode ser ausência de um cromossomo X (cariótipo 45,X), mosaicismo cromossômico (cariótipo 45,X/46,XX), além de outras anomalias estruturais do cromossomo X. As anormalidades típicas da síndrome de Turner incluem baixa estatura, disgenesia gonadal que leva a um quadro de falha do desenvolvimento puberal e infertilidade, além de uma série de outras alterações fenotípicas como pescoço alado, linha posterior de implantação dos cabelos baixa, fácies típica, tórax alargado com aumento da distância entre os mamilos, linfedema e cúbito valgo. Associa - se frequentemente a quadros de tireoidite autoimune com ou sem hipotireoidismo, anormalidades renais e cardiovasculares que variam de malfor

# Criar strings para indexação

# Indexar textos

In [7]:
# Indexar os textos no vector store
os.environ["TOKENIZERS_PARALLELISM"] = "true"
modelo_embedding = HuggingFaceInstructEmbeddings(model_name=modelo, cache_folder=diretorio_cache_modelo)
vectorstore = FAISS.from_texts(texts=[dic['topic_content'] for sublist in lista_embeddings for dic in sublist], embedding=modelo_embedding, metadatas=[dic['metadata'] for sublist in lista_embeddings for dic in sublist])

print("Textos vetorizados.")

load INSTRUCTOR_Transformer
max_seq_length  512
Textos vetorizados.


# Testar embeddings

In [41]:
# Lista de perguntas para tese
lista_query = []
lista_query.append("Quais são os critérios de inclusão do PCDT de Doença de Alzheimer")
lista_query.append("Quais são os critérios de exclusão do PCDT de Síndrome de Turner?")
lista_query.append("Elenque para mim as etapas para o diagnóstico de um paciente com Síndrome de turner?") #

# Adequar as perguntas para o padrão de treinamento do modelo
lista_query_prefix = []
for query in lista_query:
    lista_query_prefix.append("query: "+ query)
   
# Visualizar o resultado
for i, query in enumerate(lista_query_prefix):
    print(f"Pergunta: {lista_query[i]}")
    docs_scores = vectorstore.similarity_search_with_score(query, k=5)
    lista_docs_ordenada = sorted(docs_scores, key=lambda x: x[1])

    for i, _ in enumerate(lista_docs_ordenada): 
        print(f"\t{i+1}) Score: {lista_docs_ordenada[i][1]:.5f} - PCDT: {lista_docs_ordenada[i][0].metadata['source']} - Tópico: {lista_docs_ordenada[i][0].metadata['topic']}")
    print("")

Pergunta: Quais são os critérios de inclusão do PCDT de Doença de Alzheimer
	1) Score: 0.15820 - PCDT: Doença de Alzheimer - Tópico: BENEFÍCIOS ESPERADOS
	2) Score: 0.17904 - PCDT: Doença de Alzheimer - Tópico: CRITÉRIOS DE INCLUSÃO
	3) Score: 0.18139 - PCDT: Doença de Alzheimer - Tópico: CRITÉRIOS DE INCLUSÃO
	4) Score: 0.19642 - PCDT: Doença de Alzheimer - Tópico: MONITORIZAÇÃO
	5) Score: 0.20267 - PCDT: Doença de Alzheimer - Tópico: DIAGNÓSTICO

Pergunta: Quais são os critérios de exclusão do PCDT de Síndrome de Turner?
	1) Score: 0.14940 - PCDT: Síndrome de Turner - Tópico: CRITÉRIOS DE EXCLUSÃO
	2) Score: 0.15241 - PCDT: Síndrome de Turner - Tópico: CRITÉRIOS DE INCLUSÃO
	3) Score: 0.18446 - PCDT: Síndrome de Turner - Tópico: INTRODUÇÃO
	4) Score: 0.19729 - PCDT: Síndrome de Turner - Tópico: DIAGNÓSTICO
	5) Score: 0.19762 - PCDT: Síndrome de Turner - Tópico: CLASSIFICAÇÃO ESTATÍSTICA INTERNACIONAL DE DOENÇAS

Pergunta: Elenque para mim as etapas para o diagnóstico de um paciente c

# Salvar vector store

In [43]:
# Salvar o vector store em um arquivo
path_vectorstore = './vectorstore/pcdt.index'
vectorstore.save_local(path_vectorstore)

print("Vector store salvo em disco!")

# Recarregar o vectostore
vectorstore = FAISS.load_local(path_vectorstore, modelo_embedding)
print("Vector store carregado!")

Vector store salvo em disco!
Vector store carregado!
